In [1]:
import numpy as np
from powerbox.tools import get_power
from astropy.cosmology import Planck18 as cosmo
import astropy.units as un

In [2]:
def compute_power(
   box,
   length,
   n_psbins,
   log_bins=True,
   ignore_kperp_zero=True,
   ignore_kpar_zero=False,
   ignore_k_zero=False,
):
    # Determine the weighting function required from ignoring k's.
    k_weights = np.ones(box.shape, dtype=int)
    n0 = k_weights.shape[0]
    n1 = k_weights.shape[-1]

    if ignore_kperp_zero:
        k_weights[n0 // 2, n0 // 2, :] = 0
    if ignore_kpar_zero:
        k_weights[:, :, n1 // 2] = 0
    if ignore_k_zero:
        k_weights[n0 // 2, n0 // 2, n1 // 2] = 0

    res = get_power(
        box,
        boxlength=length,
        bins=n_psbins,
        bin_ave=False,
        get_variance=True,
        log_bins=log_bins,
        k_weights=k_weights,
    )

    res = list(res)
    k = res[1]
    if log_bins:
        k = np.exp((np.log(k[1:]) + np.log(k[:-1])) / 2)
    else:
        k = (k[1:] + k[:-1]) / 2

    res[1] = k
    return res
def chunk_indices_fre(params, start_redshift, cell_length):
    z=[]
    f=[]
    l=[]
    ll=[]
    fre21,df,fre_start,fre_low = params
    for i in range(int((fre_start-fre_low)/df+1)):
        fre=fre_low+i*df
        z.append(fre21/fre-1)
    z.reverse()
    for i in range(int((fre_start-fre_low)/df)):
        l.append((cosmo.comoving_distance(z[i+1])-cosmo.comoving_distance(start_redshift))/(cell_length*un.Mpc))
        ll.append(round(np.array(l)[i]))
        lll = [num for num in ll if num > 0]
    return lll
def chunk_indices_redshift(params, start_redshift, cell_length):
    z=[]
    l=[]
    ll=[]
    zup,zlow,dz = params
    for i in range(int((zup-zlow)/dz)):
        z.append(zlow+i*dz)
    for i in range(int((zup-zlow)/dz)-1):
        l.append((cosmo.comoving_distance(z[i+1])-cosmo.comoving_distance(start_redshift))/(cell_length*un.Mpc))
        ll.append(round(np.array(l)[i]))
        lll = [num for num in ll if num > 0]
    return lll
def powerspectra(brightness_temp, n_psbins, params, cell_length, min_k=0.1, max_k=1, logk=True):
    data = []
    #params=fre21,df,fre_start,fre_low
    indices =chunk_indices(params)

    #if len(chunk_indices) > nchunks:
        #chunk_indices = chunk_indices[:-1]
    #chunk_indices.append(len(zslice))

    for i in range(len(indices)-1):
        start = indices[i]
        end = indices[i + 1]
        chunklen = (end - start) * cell_length #the only not auto parameter it is cell size

        power, k, var = compute_power(
            brightness_temp[:, :, start:end],
            (BOX_LEN, BOX_LEN, chunklen),
            n_psbins,
            log_bins=logk,
        )
        data.append({"k": k, "delta": power * k ** 3 / (2 * np.pi ** 2),"var":var**(1/2)* k ** 3 / (2 * np.pi ** 2)})
    return data

In [3]:
param=(1420,8,350,50)

In [4]:
indices =chunk_indices_fre(param, 4, 1.5)

In [5]:
indices

[17,
 83,
 151,
 220,
 290,
 361,
 434,
 508,
 583,
 659,
 737,
 817,
 898,
 981,
 1066,
 1153,
 1242,
 1334,
 1428,
 1525,
 1625,
 1728,
 1835,
 1946,
 2062,
 2183,
 2310,
 2444,
 2587,
 2739]

In [6]:
param = (28,3.01,0.5)
ind=chunk_indices_redshift(param, 4, 1.5)

In [7]:
ind

[5,
 223,
 413,
 582,
 732,
 867,
 989,
 1100,
 1202,
 1296,
 1382,
 1463,
 1538,
 1608,
 1673,
 1735,
 1793,
 1848,
 1899,
 1948,
 1995,
 2039,
 2082,
 2122,
 2160,
 2197,
 2233,
 2266,
 2299,
 2330,
 2360,
 2389,
 2417,
 2444,
 2470,
 2495,
 2519,
 2543,
 2566,
 2588,
 2609,
 2630,
 2650,
 2670,
 2689,
 2708,
 2726]